## Importing the required libraries

In [97]:
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow.keras.optimizers as optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Import the dataset

In [98]:
bodyfat = pd.read_csv('bodyfat.csv')
bodyfat.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


## Scaling the data using MinMaxScaler

In [99]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bodyfat)
bodyfat_scaled = pd.DataFrame(scaled_features, index=bodyfat.index, columns=bodyfat.columns)
bodyfat_scaled.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,0.665496,0.258947,0.016949,0.146127,0.792746,0.253731,0.242531,0.200762,0.151515,0.294264,0.267081,0.189189,0.356436,0.460432,0.232143
1,0.792801,0.128421,0.000000,0.223789,0.886010,0.368159,0.251318,0.172808,0.218501,0.286783,0.267081,0.290541,0.282178,0.568345,0.428571
2,0.407375,0.532632,0.000000,0.145105,0.761658,0.144279,0.289982,0.235070,0.226475,0.309227,0.366460,0.331081,0.198020,0.302158,0.142857
3,0.703248,0.218947,0.067797,0.270795,0.886010,0.313433,0.395431,0.216010,0.258373,0.321696,0.267081,0.250000,0.376238,0.604317,0.428571
4,0.342406,0.604211,0.033898,0.268751,0.865285,0.164179,0.316344,0.388818,0.269537,0.399002,0.571429,0.331081,0.366337,0.482014,0.339286


## Splitting the dataset

In [100]:
Y = bodyfat_scaled['BodyFat']
X = bodyfat_scaled.drop(columns = 'BodyFat', axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

## Storing Learning Rates, Epochs, Activations and Regularizations to train

In [101]:
learning_rate = [1, 0.3, 0.1, 0.01,0.03,0.001,0.0001,0.00001]
epochs = [10,20,30,40]
activations = ['sigmoid', 'relu']
regularizations = [None,'l1', 'l2']
train_values, test_values, loss_values, val_loss_values = [], [], [], []

## 1 - Hidden Layer Model

In [102]:
def One_Layer_Model(epochs, lr, reg, activation):
  train_values = {}
  test_values = {}
  loss = {}
  val_loss = {}
  if reg == None :
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape = 14 , name = 'InputLayer'))
      model.add(tf.keras.layers.Dense(128, activation = activation , name="HiddenLayer1"))
      model.add(tf.keras.layers.Dense(1, activation = 'linear', name='OutputLayer'))
      
      model.compile(loss="mean_squared_error", optimizer=optimizers.Adam(learning_rate = lr), metrics=[tf.keras.metrics.RootMeanSquaredError(name = 'rmse')])

      history = model.fit(X_train, Y_train, batch_size = 8, epochs=epochs, validation_split=0.15, verbose = 0)

      test = model.evaluate(X_test, Y_test, verbose = 2)

      train_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['rmse']   
      test_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = test[1]
      loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['loss']
      val_loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['val_loss']
  
  elif reg == 'l1' or reg == 'l2':
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape = (14,), name = 'InputLayer'))
      model.add(tf.keras.layers.Dense(128, activation = activation, name="HiddenLayer1", kernel_regularizer = reg))
      model.add(tf.keras.layers.Dense(1, activation = 'linear', name='OutputLayer'))
      
      model.compile(loss="mean_squared_error", metrics=[tf.keras.metrics.RootMeanSquaredError(name = 'rmse')])

      history = model.fit(X_train, Y_train, epochs=epochs, validation_split=0.15, verbose = 0)

      test = model.evaluate(X_test, Y_test, verbose = 2)

      train_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['rmse']   
      test_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = test[1]
      loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['loss']
      val_loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['val_loss']

  return train_values, test_values, loss, val_loss

## Training Using No Regularization and L1, L2 Regularization 

In [103]:
for lr in learning_rate:
  for epoch in epochs:
    for activation in activations:
      for reg in regularizations:
        train_val , test_val, loss, val_loss = One_Layer_Model(epochs = epoch, lr = lr, activation = activation, reg = reg)
        train_values.append(train_val)
        test_values.append(test_val)
        loss_values.append(loss)
        val_loss_values.append(val_loss)

2/2 - 0s - loss: 0.0184 - rmse: 0.1358 - 19ms/epoch - 9ms/step
2/2 - 0s - loss: 0.8771 - rmse: 0.1401 - 23ms/epoch - 12ms/step
2/2 - 0s - loss: 0.1334 - rmse: 0.1177 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0170 - rmse: 0.1302 - 19ms/epoch - 9ms/step
2/2 - 0s - loss: 0.9489 - rmse: 0.1190 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 0.1441 - rmse: 0.0824 - 19ms/epoch - 10ms/step
2/2 - 0s - loss: 0.0189 - rmse: 0.1373 - 20ms/epoch - 10ms/step
2/2 - 0s - loss: 0.4022 - rmse: 0.1230 - 28ms/epoch - 14ms/step
2/2 - 0s - loss: 0.0489 - rmse: 0.0951 - 19ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0159 - rmse: 0.1260 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.3158 - rmse: 0.0819 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0604 - rmse: 0.0687 - 20ms/epoch - 10ms/step
2/2 - 0s - loss: 0.0177 - rmse: 0.1329 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0434 - rmse: 0.1355 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0293 - rmse: 0.0859 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0187 - rmse: 0.1367 - 18ms/epoc

## 2 - Hidden Layers Model

In [104]:
def Two_Layer_Model(epochs, lr, reg, activation):
  train_values = {}
  test_values = {}
  loss = {}
  val_loss = {}
  if reg == None :
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape = 14 , name = 'InputLayer'))
      model.add(tf.keras.layers.Dense(128, activation = activation , name="HiddenLayer1"))
      model.add(tf.keras.layers.Dense(64, activation = activation , name="HiddenLayer2"))
      model.add(tf.keras.layers.Dense(1, activation = 'linear', name='OutputLayer'))
      
      model.compile(loss="mean_squared_error", optimizer=optimizers.Adam(learning_rate = lr), metrics=[tf.keras.metrics.RootMeanSquaredError(name = 'rmse')])

      history = model.fit(X_train, Y_train, batch_size = 8, epochs=epochs, validation_split=0.15, verbose = 0)

      test = model.evaluate(X_test, Y_test, verbose = 2)

      train_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['rmse']   
      test_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = test[1]
      loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['loss']
      val_loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['val_loss']

  elif reg == 'l1' or reg == 'l2':
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape = (14,), name = 'InputLayer'))
      model.add(tf.keras.layers.Dense(128, activation = activation, name="HiddenLayer1", kernel_regularizer = reg))
      model.add(tf.keras.layers.Dense(64, activation = activation, name="HiddenLayer2", kernel_regularizer = reg))
      model.add(tf.keras.layers.Dense(1, activation = 'linear', name='OutputLayer'))
      
      model.compile(loss="mean_squared_error", metrics=[tf.keras.metrics.RootMeanSquaredError(name = 'rmse')])

      history = model.fit(X_train, Y_train, epochs=epochs, validation_split=0.15, verbose = 0)

      test = model.evaluate(X_test, Y_test, verbose = 2)

      train_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['rmse']   
      test_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = test[1]
      loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['loss']
      val_loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['val_loss']

  return train_values, test_values, loss, val_loss

## Training Using No Regularization and L1, L2 Regularization 

In [105]:
for lr in learning_rate:
  for epoch in epochs:
    for activation in activations:
      for reg in regularizations:
        train_val , test_val, loss, val_loss = Two_Layer_Model(epochs = epoch, lr = lr, activation = activation, reg = reg)
        train_values.append(train_val)
        test_values.append(test_val)
        loss_values.append(loss)
        val_loss_values.append(val_loss)

2/2 - 0s - loss: 0.1379 - rmse: 0.3714 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 4.5277 - rmse: 0.1342 - 30ms/epoch - 15ms/step
2/2 - 0s - loss: 0.6901 - rmse: 0.1304 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0205 - rmse: 0.1431 - 24ms/epoch - 12ms/step
2/2 - 0s - loss: 3.7930 - rmse: 0.1336 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 0.4027 - rmse: 0.0761 - 23ms/epoch - 12ms/step
2/2 - 0s - loss: 0.0177 - rmse: 0.1332 - 16ms/epoch - 8ms/step
2/2 - 0s - loss: 1.2199 - rmse: 0.1431 - 22ms/epoch - 11ms/step
2/2 - 0s - loss: 0.3241 - rmse: 0.1322 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0175 - rmse: 0.1324 - 17ms/epoch - 8ms/step
2/2 - 0s - loss: 0.9315 - rmse: 0.1317 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 0.1153 - rmse: 0.0756 - 25ms/epoch - 13ms/step
2/2 - 0s - loss: 0.0186 - rmse: 0.1363 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.1031 - rmse: 0.1451 - 17ms/epoch - 8ms/step
2/2 - 0s - loss: 0.1681 - rmse: 0.1637 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0213 - rmse: 0.1461 - 18ms/epoc

## 3 - Hidden Layers Model

In [106]:
def Three_Layer_Model(epochs, lr, reg, activation):
  train_values = {}
  test_values = {}
  loss = {}
  val_loss = {}
  if reg == None :
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape = 14 , name = 'InputLayer'))
      model.add(tf.keras.layers.Dense(128, activation = activation , name="HiddenLayer1"))
      model.add(tf.keras.layers.Dense(64, activation = activation , name="HiddenLayer2"))
      model.add(tf.keras.layers.Dense(32, activation = activation , name="HiddenLayer3"))
      model.add(tf.keras.layers.Dense(1, activation = 'linear', name='OutputLayer'))
      
      model.compile(loss="mean_squared_error", optimizer=optimizers.Adam(learning_rate = lr), metrics=[tf.keras.metrics.RootMeanSquaredError(name = 'rmse')])

      history = model.fit(X_train, Y_train, batch_size = 8, epochs=epochs, validation_split=0.15, verbose = 0)

      test = model.evaluate(X_test, Y_test, verbose = 2)

      train_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['rmse']   
      test_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = test[1]
      loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['loss']
      val_loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['val_loss']

  elif reg == 'l1' or reg == 'l2':
      model = tf.keras.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape = (14,), name = 'InputLayer'))
      model.add(tf.keras.layers.Dense(128, activation = activation, name="HiddenLayer1", kernel_regularizer = reg))
      model.add(tf.keras.layers.Dense(64, activation = activation, name="HiddenLayer2", kernel_regularizer = reg))
      model.add(tf.keras.layers.Dense(64, activation = activation, name="HiddenLayer3", kernel_regularizer = reg))
      model.add(tf.keras.layers.Dense(1, activation = 'linear', name='OutputLayer'))
      model.compile(loss="mean_squared_error", metrics=[tf.keras.metrics.RootMeanSquaredError(name = 'rmse')])

      history = model.fit(X_train, Y_train, epochs=epochs, validation_split=0.15, verbose = 0)

      test = model.evaluate(X_test, Y_test, verbose = 2)

      train_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['rmse']   
      test_values['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = test[1]
      loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['loss']
      val_loss['HL : 3, reg : ' + str(reg) + ', lr : ' + str(lr) + ', epochs : '+ str(epochs) + ', activation : ' + activation] = history.history['val_loss']

  return train_values, test_values, loss, val_loss

## Training Using No Regularization and L1, L2 Regularization 

In [107]:
for lr in learning_rate:
  for epoch in epochs:
    for activation in activations:
      for reg in regularizations:
        train_val , test_val, loss, val_loss = Three_Layer_Model(epochs = epoch, lr = lr, activation = activation, reg = reg)
        train_values.append(train_val)
        test_values.append(test_val)
        loss_values.append(loss)
        val_loss_values.append(val_loss)

2/2 - 0s - loss: 0.0537 - rmse: 0.2317 - 20ms/epoch - 10ms/step
2/2 - 0s - loss: 5.6885 - rmse: 0.1419 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.6450 - rmse: 0.1682 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 20.9539 - rmse: 4.5775 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 5.7336 - rmse: 0.1189 - 17ms/epoch - 8ms/step
2/2 - 0s - loss: 0.6399 - rmse: 0.1021 - 55ms/epoch - 27ms/step
2/2 - 0s - loss: 0.0888 - rmse: 0.2980 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 1.5935 - rmse: 0.1459 - 19ms/epoch - 10ms/step
2/2 - 0s - loss: 0.2533 - rmse: 0.1322 - 22ms/epoch - 11ms/step
2/2 - 0s - loss: 20.3779 - rmse: 4.5142 - 17ms/epoch - 9ms/step
2/2 - 0s - loss: 1.6427 - rmse: 0.1318 - 16ms/epoch - 8ms/step
2/2 - 0s - loss: 0.1862 - rmse: 0.0868 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0170 - rmse: 0.1305 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.2331 - rmse: 0.1654 - 18ms/epoch - 9ms/step
2/2 - 0s - loss: 0.1181 - rmse: 0.1908 - 19ms/epoch - 9ms/step
2/2 - 0s - loss: 0.0171 - rmse: 0.1308 - 18ms/epo

## Obtaining Minimum Test RMSE

In [117]:
val = []
for value in test_values:
  val.append(list(value.values()))
val.sort()
min = val[0][0]
print(min)

0.01409811433404684


## Obtaining Index of Minimum Test RMSE

In [116]:
pos = ""
for test_value in test_values:
  for key, value in test_value.items():
      if value == min:
        pos = key
        break

In [115]:
print(pos)

HL : 3, reg : None, lr : 0.01, epochs : 30, activation : relu
